# Hyper parameter Tuning 

<b> 1. Grid Search CV </b> <br>
<b> 2. Randomized Search CV </b>

In [1]:
from sklearn.datasets import load_iris
iris = load_iris()
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [2]:
import pandas as pd
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['flower'] = iris.target
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,0
48,5.3,3.7,1.5,0.2,0
49,5.0,3.3,1.4,0.2,0
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1


In [3]:
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [5]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', C=30, gamma='auto')
model.fit(x_train, y_train)

SVC(C=30, gamma='auto')

In [6]:
model.score(x_test, y_test)

0.9777777777777777

#### Using K Fold cross validation

In [7]:
from sklearn.model_selection import cross_val_score

In [8]:
cross_val_score(SVC(kernel='linear', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [9]:
cross_val_score(SVC(kernel='rbf', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [10]:
cross_val_score(SVC(kernel='rbf', C=20, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [11]:
import numpy as np
kernels = ['rbf', 'linear']
C = [1, 10, 20]
avg_scores = {}
for k_val in kernels:
    for c_val in C:
        cv_scores = cross_val_score(SVC(kernel=k_val, C=c_val, gamma='auto'), iris.data, iris.target, cv=5)
        avg_scores[k_val + '_' + str(c_val)] = np.average(cv_scores)
        
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

### Using Grid Search CV

In [12]:
from sklearn.model_selection import GridSearchCV

# cv --> cross validation
clf = GridSearchCV(SVC(gamma='auto'), {
    'C': [1, 10, 20], 
    'kernel' : ['rbf', 'linear']
}, cv=5, return_train_score=False)

# clf.fit(iris.data, iris.target)
clf.fit(x_train, y_train)
clf.cv_results_

{'mean_fit_time': array([0.00120163, 0.00127177, 0.00040011, 0.00060101, 0.00079904,
        0.0011013 ]),
 'std_fit_time': array([0.00040284, 0.00038126, 0.00049004, 0.00049072, 0.00039952,
        0.00020217]),
 'mean_score_time': array([0.00112967, 0.0006022 , 0.00059972, 0.00040102, 0.00040069,
        0.00020008]),
 'std_score_time': array([0.00073508, 0.00049171, 0.00048967, 0.00049116, 0.00049074,
        0.00040016]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [13]:
new_df = pd.DataFrame(clf.cv_results_)
new_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001202,0.000403,0.001130,0.000735,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.952381,0.952381,0.952381,1.0,1.0,0.971429,0.023328,1
1,0.001272,0.000381,0.000602,0.000492,1,linear,"{'C': 1, 'kernel': 'linear'}",0.952381,0.952381,0.952381,1.0,1.0,0.971429,0.023328,1
2,0.000400,0.000490,0.000600,0.000490,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.857143,0.952381,0.952381,1.0,1.0,0.952381,0.052164,5
3,0.000601,0.000491,0.000401,0.000491,10,linear,"{'C': 10, 'kernel': 'linear'}",0.904762,0.952381,0.952381,1.0,1.0,0.961905,0.035635,3
4,0.000799,0.000400,0.000401,0.000491,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.857143,0.952381,0.952381,1.0,1.0,0.952381,0.052164,5
5,0.001101,0.000202,0.000200,0.000400,20,linear,"{'C': 20, 'kernel': 'linear'}",0.904762,0.952381,0.952381,1.0,1.0,0.961905,0.035635,3


In [14]:
new_df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.971429
1,1,linear,0.971429
2,10,rbf,0.952381
3,10,linear,0.961905
4,20,rbf,0.952381
5,20,linear,0.961905


In [15]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_fe

In [16]:
clf.best_score_

0.9714285714285713

In [17]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

### Using Randomized Search CV

In [18]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma='auto'), {
    'C': [1, 10, 20],
    'kernel': ['rbf', 'linear']
}, 
                        cv=5,
                        return_train_score=False,
                        n_iter=2
)

rs.fit(x_train, y_train)
pd.DataFrame(rs.cv_results_)[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,linear,0.961905
1,1,rbf,0.971429


### Now comparing the performance of the models

In [19]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [20]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params': {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1, 5, 10]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'C': [1, 10, 20]
        }
    }
}

In [21]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [22]:
df1 = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df1

,model,best_score,best_params
0,svm,0.971429,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.961905,{'n_estimators': 5}
2,logistic_regression,0.971429,{'C': 20}


## Another Task

For digits dataset in sklearn.dataset, please try following classifiers and find out the one that gives best performance. Also find the optimal parameters for that classifier.

- from sklearn import svm
- from sklearn.ensemble import RandomForestClassifier
- from sklearn.linear_model import LogisticRegression
- from sklearn.naive_bayes import GaussianNB
- from sklearn.naive_bayes import MultinomialNB
- from sklearn.tree import DecisionTreeClassifier

In [23]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [24]:
from sklearn.datasets import load_digits
digits = load_digits()
dir(digits)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

### Taking different models with different parameters --> to compare their performances

In [25]:
models_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params': {
            'C':[1, 10, 20, 30],
            'kernel': ['rbf', 'linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators':[5, 10, 20, 30]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'C': [1, 10, 20, 30]
        }
    },
    'gaussian_nb': {
        'model': GaussianNB(),
        'params': {}
    },
    'multinomial_nb': {
        'model': MultinomialNB(),
        'params': {}
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ["gini", "entropy", "log_loss"],
            'splitter' : ["best", "random"]
        }
    }
}

In [26]:
from sklearn.model_selection import GridSearchCV
model_scores = []

for model_name, mp in models_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(digits.data, digits.target)
    model_scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [27]:
new_model_df = pd.DataFrame(model_scores, columns=['model', 'best_score', 'best_params'])
new_model_df

,model,best_score,best_params
0,svm,0.947697,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.932128,{'n_estimators': 30}
2,logistic_regression,0.922114,{'C': 1}
3,gaussian_nb,0.806928,{}
4,multinomial_nb,0.870350,{}
5,decision_tree,0.804169,"{'criterion': 'entropy', 'splitter': 'random'}"
